In [ ]:
# !wget https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
# !tar xf spark-3.5.3-bin-hadoop3.tgz
# !rm spark-3.5.3-bin-hadoop3.tgz

--2025-03-04 10:24:54--  https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189
connected. to archive.apache.org (archive.apache.org)|65.108.204.189|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 400864419 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.3-bin-hadoop3.tgz’

spark-3.5.3-bin-had 100%[===================>] 382.29M   255KB/s    in 33m 32s 

2025-03-04 10:58:26 (195 KB/s) - ‘spark-3.5.3-bin-hadoop3.tgz’ saved [400864419/400864419]



In [15]:
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "spark-3.5.3-bin-hadoop3"

In [16]:
!cd ~/CS/CovidComparison
!pwd

/Users/duncanbeauchamp/CS/CovidComparison


In [17]:
# !pip install -q findspark
import findspark
findspark.init()

In [ ]:
#Check how many cores are available
import multiprocessing
print(multiprocessing.cpu_count())

8


In [ ]:
#Start the Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CSV Loader") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

In [36]:
reddit_df = spark.read.csv("preprocessed_reddit.csv", header=True, inferSchema=True, multiLine=True, escape='"')

reddit_df.show()


[Stage 44:>                                                         (0 + 1) / 1]

+-------+--------------------+-----------+--------------------+-----+
|     id|      subreddit.name|created_utc|                body|score|
+-------+--------------------+-----------+--------------------+-----+
|hi1vsag|                nova| 1635206399|When you schedule...|    2|
|hi1vs7i|           vancouver| 1635206397|Didn't stop price...|   32|
|hi1vs5n|            pregnant| 1635206397|I’m just waiting ...|    1|
|hi1vs5v|            startrek| 1635206397|*The first duty o...|    1|
|hi1vs0l|             ontario| 1635206395|Compare BC to Ont...|   -2|
|hi1vrtk|       entertainment| 1635206393|Sheeran didn't me...|    2|
|hi1vrsk|          conspiracy| 1635206392|I see a lot of po...|    1|
|hi1vrnr|  lockdownskepticism| 1635206391|And think about h...|   29|
|hi1vrpy|                nova| 1635206391|Unfortunately the...|    2|
|hi1vrou|         antivaxxers| 1635206391|I don’t know anyo...|    1|
|hi1vrg2|          conspiracy| 1635206388|BS. Not a fear ca...|    1|
|hi1vre5|      toron

In [ ]:
#Remove new lines from the body column, we used pyspark dfs ONLY to remove newline characters, not to do any big data processing
from pyspark.sql.functions import regexp_replace

reddit_df_cleaned = reddit_df.withColumn("body", regexp_replace("body", "[\r\n]", " ")).select("id", "body")

reddit_df_cleaned.take(2)

[Row(id='hi1vsag', body='When you scheduled your booster with CVS does it just give you the option of Vaccines: COVID-19 (Vaccine brand) or does it specifically say booster'),
 Row(id='hi1vs7i', body="Didn't stop prices there though. New Zealand and Canada grew at about the same rate through COVID.   I agree that non-resident ownership should be stopped, but it's also probably not going to change anything.")]

In [38]:
reddit_df_cleaned.coalesce(1).write.csv("./reddit_full_cleaned", header=True)

In [ ]:
twitter_df = spark.read.csv("preprocessed_twitter.csv", header=True, inferSchema=True, multiLine=True, escape='"')

twitter_df.show()

[Stage 36:>                                                         (0 + 1) / 1]

+-------------------+-------------------+---------------+--------------------+----+--------------------+-------------+--------------------+------------+---------------+--------+
|          status_id|         created_at|    screen_name|                text|lang|              source|retweet_count|reply_to_screen_name|country_code|followers_count|verified|
+-------------------+-------------------+---------------+--------------------+----+--------------------+-------------+--------------------+------------+---------------+--------+
|1244051646071611394|2020-03-29 00:00:00|   IMSS_SanLuis|Ante cualquier en...|  es|           TweetDeck|            0|                NULL|        NULL|           1008|   false|
|1244051645039706112|2020-03-29 00:00:00|     intrac_ccs|#ATENCIÓN En el T...|  es|           TweetDeck|            1|                NULL|        NULL|             90|   false|
|1244051645975191557|2020-03-29 00:00:00|       rlieving|“People are just ...|  en|           TweetDeck|      

In [ ]:
#Same as above, remove new lines from the text column
twitter_df_cleaned = twitter_df.withColumn("text", regexp_replace("text", "[\r\n]", " ")).select("status_id", "lang", "text")

twitter_df_cleaned.take(2)

[Row(status_id=1244051646071611394, lang='es', text='Ante cualquier enfermedad respiratoria, no te automediques. #PrevenciónCoronavirus #Coronavirus #COVID19 https://t.co/IgB2izYi1p'),
 Row(status_id=1244051645039706112, lang='es', text='#ATENCIÓN En el Terminal Nuevo Circo se implementan medidas como el uso de la mascarilla por parte de usuari@s y conductores que hacen vida en dichas instalaciones, parte de los esfuerzos unificados para impedir la propagación del #Coronavirus #28Marzo @NicolasMaduro  @ErikaPSUV https://t.co/yD8g0EAcQz')]

In [34]:
twitter_df_cleaned.coalesce(1).write.csv("./twitter_full_cleaned", header=True)